**E-Commerce Conversion Rate: Simple A/B Testing**

In [41]:
### Importing necessary modules: 
import pandas as pd 
from statsmodels.stats.proportion import proportions_ztest 

**Data review and preparation**

In [42]:
df_0 = pd.read_csv('ab_test.csv')  # Main A/B test dataset
df_1 = pd.read_csv('countries_ab.csv')  # Dataset containing user countries

df_0.head()  # Display the first few rows of the main dataset

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [43]:
df_1.head()  # Display the first few rows of the dataset with user countries 

,id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK


In [44]:
df = pd.merge(df_0, df_1, on='id')  # Merge main dataset with user countries by primary key 'id' and save the result to 'df'
df.head()  # Display the first few rows of the merged dataset

,id,time,con_treat,page,converted,country
0,851104,11:48.6,control,old_page,0,US
1,804228,01:45.2,control,old_page,0,US
2,661590,55:06.2,treatment,new_page,0,US
3,853541,28:03.1,treatment,new_page,0,US
4,864975,52:26.2,control,old_page,1,US


Display general information about the DataFrame, including column names, data types, and missing values.

In [45]:
print('DataFrame shape:', df.shape)
print('____________________________') 
print('Information about data:')
print(df.info()) 
print('____________________________') 
print('Missing values:') 
print(df.isna().sum())

DataFrame shape: (294478, 6)
____________________________
Information about data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
 5   country    294478 non-null  object
dtypes: int64(2), object(4)
memory usage: 13.5+ MB
None
____________________________
Missing values:
id           0
time         0
con_treat    0
page         0
converted    0
country      0
dtype: int64


I want to compare the number of rows with the number of unique user IDs.

In [46]:
df.nunique() # display the number of unique values for each column

id           290584
time          35993
con_treat         2
page              2
converted         2
country           3
dtype: int64

There are duplicates in the 'id' column. I will drop these duplicates because they might be mistakes. Dropping some rows is not critical, as the dataset is quite large.

In [47]:
df_ = df.drop_duplicates(subset='id') 
print('Number of rows without duplicates -', df_.shape[0])

Number of rows without duplicates - 290584


Then it's important to research whether the user groups match the page version.

In [48]:
mistakes = df_[ 
    (
        ((df_['con_treat'] == 'control') & (df_['page'] == 'new_page')) | 
        ((df_['con_treat'] == 'treatment') & (df_['page'] == 'old_page'))
    )
]
print('Number of wrong rows -', mistakes.shape[0])
mistakes.tail()  # show how it looks like

Number of wrong rows - 2044


,id,time,con_treat,page,converted,country
282645,717723,43:58.1,treatment,old_page,0,US
282950,778907,54:16.6,control,new_page,0,US
285987,914482,55:40.1,treatment,old_page,0,UK
286353,767924,48:53.4,treatment,old_page,0,US
287469,921581,11:58.6,treatment,old_page,0,US


In [49]:
df_clear = df_[ # creating clean DataFrame 
    (
        ((df_['con_treat'] == 'control') & (df_['page'] != 'new_page')) | 
        ((df_['con_treat'] == 'treatment') & (df_['page'] != 'old_page')) 
    )
]
print('Number of rows without mistakes -', df_clear.shape[0]) 

Number of rows without mistakes - 288540


Besides testing the full dataset, I would like to test samples separated by country.

In [50]:
samples = [df_clear[df_clear['country'] == i] for i in df_clear['country'].unique()]  # Create a list of samples, one for each country

for sample in samples:  # Show the number of users in each country
    print(sample.shape[0], '- number of users from', sample['country'].unique()[0])

202185 - number of users from US
14394 - number of users from CA
71961 - number of users from UK


**A/B Testing**

In [51]:
def ztest(data, sample): 
    ''' 
    Function to perform Z-test for any case: 
    general population or country sample
    '''
    
    H0 = "No difference between groups"  # null hypothesis
    H1 = "Groups have differences"  # alternative hypothesis
    alpha_ = 0.05  # critical value for the probability of the Z-statistic
    
    # Separating groups
    control_group = data[data['con_treat'] == 'control']
    test_group = data[data['con_treat'] == 'treatment']
    
    # Summing conversions for each group
    control_conversions = control_group['converted'].sum()
    test_conversions = test_group['converted'].sum()
    
    # Finding the number of observations
    control_group_total_conversions = control_group.shape[0]
    test_group_total_conversions = test_group.shape[0]

    conversions = [control_conversions, test_conversions]
    totals = [control_group_total_conversions, test_group_total_conversions]
    
    # Perform Z-test
    z_stat, p_value = proportions_ztest(conversions, totals)
    
    result = []  # empty list for results based on p_value
    if p_value < alpha_:
        result.append(H1)
    elif p_value >= alpha_:
        result.append(H0)
    
    # Return the results
    if sample == 'general':
        return pd.DataFrame({
            'sample': ['general_population'],
            'z_stat': [z_stat],
            'p_value': [p_value],
            'A/B test result': result
        })
    elif sample == 'sample':
        return pd.DataFrame({
            'sample': [data['country'].unique()[0]],
            'z_stat': [z_stat],
            'p_value': [p_value],
            'A/B test result': result
        })
    else:
        raise ValueError("Sample should be 'general' or 'sample'")

general_result = ztest(data=df_clear, sample='general') # Test for all data

results = []
for s in samples:  # Tests for samples
    r = ztest(data=s, sample='sample')
    results.append(r)

results.append(general_result)  # Combine all results

# Convert list to DataFrame and display the result
all_results = pd.concat([j for j in results], axis=0)
all_results


,sample,z_stat,p_value,A/B test result
0,US,1.467193,0.142324,No difference between groups
0,CA,1.320795,0.186570,No difference between groups
0,UK,-0.455738,0.648578,No difference between groups
0,general_population,1.294237,0.195584,No difference between groups


**Conclusion**

There is no statistical difference between the conversion rates of the old and new pages, so we can either continue using the old page or keep working to improve the new one.